<a href="https://colab.research.google.com/github/aravindskumar98/DL/blob/main/ClassifierFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader


In [4]:
import numpy as np
data = np.load('Q4_data.npz')

In [5]:
X = data['X']
y = data['Y']

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [7]:
num_epochs = 200
batch_size = len(X)
learning_rate = 1

In [8]:
input_size = 2
hidden_size = 40
num_classes = 4

In [9]:
X = torch.Tensor(X)
Y = torch.Tensor(y)
# Y = F.one_hot(Y.to(torch.int64), num_classes=4)
dataset = TensorDataset(X,Y) 
my_dataloader = DataLoader(dataset,batch_size = batch_size, shuffle = False)

In [10]:
class NeuralNet(nn.Module):

  def __init__(self, input, hidden, num_classes):
    super(NeuralNet,self).__init__()
    self.l1 = nn.Linear(input,hidden)
    self.l2 = nn.Linear(hidden,num_classes)
  
  def forward(self,x):
    out = self.l2(F.relu(self.l1(x)))
    return out
  

In [11]:

model = NeuralNet(input_size, hidden_size, num_classes).to(device) 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [12]:
total_steps = len(my_dataloader)
for epoch in range(num_epochs):
  for i,(inputData,labels) in enumerate(my_dataloader):
    inputData = inputData.to(device)
    labels = labels.to(device)
    outputs = model(inputData)
    # outputs = model(inputData)
    loss = criterion(outputs,labels.long())

    #backwards
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch%10 == 0:
      print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{total_steps}, loss = {loss.item():.4f}')

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for inputData, labels in my_dataloader:
    inputData = inputData.to(device)
    labels = labels.to(device)
    outputs = model(inputData)

    # value, index is returned
    _,predictions = torch.max(outputs,1)
    n_samples+=labels.shape[0]
    n_correct+=(predictions == labels).sum().item() 

acc = 100.0*n_correct/n_samples

print("accuracy", acc)

epoch 1/200, step 1/1, loss = 1.6169
epoch 11/200, step 1/1, loss = 9.5477
epoch 21/200, step 1/1, loss = 2.7251
epoch 31/200, step 1/1, loss = 1.8017
epoch 41/200, step 1/1, loss = 0.8008
epoch 51/200, step 1/1, loss = 0.3353
epoch 61/200, step 1/1, loss = 1.1281
epoch 71/200, step 1/1, loss = 0.3703
epoch 81/200, step 1/1, loss = 0.6105
epoch 91/200, step 1/1, loss = 0.1258
epoch 101/200, step 1/1, loss = 0.1142
epoch 111/200, step 1/1, loss = 0.0902
epoch 121/200, step 1/1, loss = 0.0418
epoch 131/200, step 1/1, loss = 0.0376
epoch 141/200, step 1/1, loss = 0.0349
epoch 151/200, step 1/1, loss = 0.0332
epoch 161/200, step 1/1, loss = 0.0315
epoch 171/200, step 1/1, loss = 0.0300
epoch 181/200, step 1/1, loss = 0.0286
epoch 191/200, step 1/1, loss = 0.0272
accuracy 100.0
